In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from scipy import stats
from scipy import optimize
from bioinfokit.analys import stat
import statsmodels.api as sm
import numpy as np
from scipy.stats import mannwhitneyu
from scipy.stats import wilcoxon
from scipy.stats import kruskal

#### Ler dados do dataframe

In [ ]:
cols = ['NumeroFatura', 'CodigoProduto', 'Descricao',
        'Quantidade', 'DataFatura', 'PrecoUnitario',
        'IDCliente', 'Pais']
# Ler arquivoa CSV
# df1 = pd.read_csv('ano_2009-2010.csv', encoding='latin1')
df = pd.read_csv('ano_2010-2011.csv', encoding='latin1')

# Concatenar os dois DataFrames
# df = pd.concat([df1, df2], ignore_index=True)
df.columns = cols

# Imprimir o DataFrame resultante
df

#### Realiza limpeza dos dados

In [ ]:
# Exclui todas faturas canceladas
df = df[~df['NumeroFatura'].str.startswith(('C', 'c'))]

# Exclui linhas que tenham valor e quantidades menores ou igual a zero
df = df[df["Quantidade"] > 0]
df = df[df["PrecoUnitario"] > 0.0]

df

#### Gera Dataframe com agrupamento

In [ ]:
# cols_to_group = ['CodigoProduto', 'Pais']
cols_to_group = ['Pais']
df_final = df.groupby(cols_to_group).agg({"Quantidade": "mean", "PrecoUnitario": "mean"})
df_final = df_final.reset_index()
df_final = df_final.dropna()
df_final

In [ ]:
# Gráfico de barras para comparar o somat´0rio das quantidades entre os países
plt.figure(figsize=(12, 6))
sns.barplot(x='Pais', y='Quantidade', data=df_final, errorbar=None)
plt.xticks(rotation=90)
plt.title('Somatorio das Quantidades por País')
plt.show()

In [ ]:
# Gráfico de barras para comparar as médias dos preços entre os países
plt.figure(figsize=(12, 6))
sns.barplot(x='Pais', y='PrecoUnitario', data=df_final, errorbar=None)
plt.xticks(rotation=90)
plt.title('Médias dos Preços por País')
plt.show()

#### Aferindo Normalidade

In [ ]:
stats.probplot(df_final["Quantidade"], dist="norm", plot=plt)
plt.show()
stats.probplot(df_final["PrecoUnitario"], dist="norm", plot=plt)
plt.show()


#### Prepara para aferição teste ANOVA

In [ ]:
df_melt = pd.melt(df_final.drop(cols_to_group, axis=1))
df_melt

In [ ]:
# Ordinary Least Squares (OLS) model
model = ols('value ~ variable', data=df_melt).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
anova_table

In [ ]:
res = stat()
res.anova_stat(df_melt, res_var='value', anova_model='value ~ variable')
res.anova_summary

In [ ]:
df_melt['variable'] = df_melt['variable'].replace('Quantidade', '1')
df_melt['variable'] = df_melt['variable'].replace('PrecoUnitario', '2')
df_melt['variable'].unique()

In [ ]:
res = stat()
res.tukey_hsd(df_melt, res_var='value', xfac_var='variable', anova_model='value ~ variable')
res.tukey_summary

#### Avaliação dos residuos

In [ ]:
sm.qqplot(res.anova_std_residuals, line='45')
plt.xlabel("Theoretical Quantiles")
plt.ylabel("Standardized Residuals")
plt.show()

In [ ]:
plt.hist(res.anova_model_out.resid, bins='auto', histtype='bar', ec='k') 
plt.xlabel("Residuals")
plt.ylabel('Frequency')
plt.xlim(-1000, 1000)
plt.ylim(0, 1000)
plt.show()

In [ ]:
w, pvalue = stats.shapiro(model.resid)
print('Statistics= %.3f, pvalue=%.3f' % (w, pvalue))
# interpret
alpha = 0.05
if pvalue > alpha:
	print('Resíduo com Distribuição Normal (Não rejeita H0)')
else:
	print('Diferente da Normal (rejeita H0)')

In [ ]:
from scipy import stats

# perform the Kolmogorov-Smirnov test on the residuals
k_statistic, pvalue = stats.kstest(model.resid, 'norm')

# print the results and interpret
alpha = 0.05
if pvalue > alpha:
    print('Resíduo com Distribuição Normal (Não rejeita H0)')
else:
    print('Diferente da Normal (rejeita H0)')

print('K-S Statistic= %.3f, pvalue=%.3f' % (k_statistic, pvalue))

In [ ]:
w, pvalue = stats.bartlett(df_final['Quantidade'], df_final['PrecoUnitario'])
print('Statistics= %.3f, pvalue=%.3f' % (w, pvalue))
# interpret
alpha = 0.05
if pvalue > alpha:
	print('Same Variance (Não rejeita H0)')
else:
	print('Different Variance (rejeita H0)')

In [ ]:
res = stat()
res.bartlett(df_melt, res_var='value', xfac_var='variable')
res.bartlett_summary

#### Não paramétrico Testes -> Mann-Whitney U Test

In [ ]:
stats, p = mannwhitneyu(df_final['Quantidade'], df_final['PrecoUnitario'])
print('Statistics= %.3f, p=%.3f' % (stats, p))
# interpret
alpha = 0.05
if p > alpha:
	print('Same distribution (Não rejeita H0)')
else:
	print('Different distribution (rejeita H0)')

In [ ]:
stats, p = wilcoxon(df_final['Quantidade'], df_final['PrecoUnitario'])
print('Statistics= %.3f, p=%.3f' % (stats, p))
# interpret
alpha = 0.05
if p > alpha:
	print('Same distribution (Não rejeita H0)')
else:
	print('Different distribution (rejeita H0)')